In [1]:
from keras.models import load_model
import pickle
import tensorflow as tf
from music21 import converter, instrument, note, chord, stream

In [2]:
def generate(model, input_notes, filename, latent_dim = 500):
    # Get pitch names and store in a dictionary
    notes = input_notes
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # Use random noise to generate sequences
    noise = tf.random.normal([1, latent_dim])
    #noise = np.random.normal(0, 1, (1, latent_dim))
    predictions = model.predict(noise)
    #for normalized -1 to 1
    p = len(pitchnames)/2
    pred_notes = [x*p+p for x in predictions[0]]
    #for normalized 0 to 1
    #p = len(pitchnames)
    #pred_notes = [x*p for x in predictions[0]]
    pred_notes = [int_to_note[int(x)] for x in pred_notes]
    print('pred_notes:',pred_notes)
    create_midi(pred_notes, filename)

In [3]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                output_notes.append(instrument.Piano())                
                new_note = note.Note(int(current_note))
                
                #new_note.storedInstrument = instrument.StringInstrument()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            output_notes.append(instrument.Piano())
            new_note = note.Note(pattern)
            new_note.offset = offset            
            #new_note.storedInstrument = instrument.StringInstrument()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

In [4]:
filename = 'notes_chopin'
with open('notes/'+filename, 'rb') as filepath:
    notes_chopin = pickle.load(filepath)

In [5]:
notes_chopin

['0.4',
 '4.7',
 '0.4.7',
 '0.4.7',
 '0.4.7',
 '0.4.7',
 '0.4.7',
 'G5',
 'E5',
 'E5',
 'D5',
 'C5',
 'D5',
 '4.7',
 'C5',
 'D5',
 'C5',
 '4.7',
 'D5',
 'C5',
 'B4',
 'C5',
 'A5',
 'F5',
 'E5',
 'D5',
 '5.9',
 'E5',
 'D5',
 'E5',
 'D5',
 'E5',
 '5.9',
 'D5',
 'C5',
 'D5',
 'F5',
 'D5',
 'E5',
 'D5',
 '7.11',
 'G5',
 '7.11',
 '5.7.11',
 'G5',
 '7.0',
 'F5',
 'G5',
 'F5',
 'G5',
 'F5',
 'E5',
 'F5',
 '0.4',
 'G4',
 '0.4.7',
 '0.4.7',
 'C2',
 'E2',
 '0.4.7',
 'G2',
 '0.4.7',
 'E2',
 '0.4.7',
 '0.4.7',
 'G4',
 'E4',
 'E4',
 'D4',
 'C4',
 'D4',
 '0.4.7',
 'C4',
 'D4',
 'C4',
 'D4',
 '0.4.7',
 'C4',
 'B3',
 'C4',
 'A4',
 'F4',
 'E4',
 'D4',
 'E4',
 '2.5.9',
 'D4',
 'E4',
 'D4',
 'E4',
 '2.5.9',
 'D4',
 'C4',
 'D4',
 'F4',
 'D4',
 'C4',
 'B3',
 '7',
 'A4',
 'G3',
 'G2',
 '5.7',
 'G2',
 'G4',
 '7',
 'F4',
 'G4',
 'F4',
 'G4',
 'F4',
 'E4',
 'F4',
 '4.9',
 'C5',
 'D5',
 'E5',
 'F5',
 'G5',
 'C4',
 '0',
 'E3',
 'G4',
 'G3',
 'E4',
 'G5',
 'C4',
 'G4',
 'G3',
 '7.0',
 'E3',
 'C2',
 'C3',
 'G4',
 

In [10]:
model_name = 'model_gen_chopin.hdf5'
model =  load_model('model/'+ model_name)   

In [16]:
generate(model, notes_chopin, 'gen_music/gans_chopin4', latent_dim = 500)

pred_notes: ['F6', 'C6', 'B-4', 'G6', 'C#3', 'G2', 'F#5', 'B-5', 'D4', 'G#3', 'F#1', 'F4', 'E2', 'C3', 'F4', 'F#4', 'G#4', 'C#4', 'D4', 'F#2', 'F#6', 'F4', 'E6', '9.11.2', '9.11.2', 'D3', '9.10', 'C5', '9.10.2', '9.11.0', '7.11.0', 'E-4', 'E3', 'D7', '9.0.3.5', 'B2', 'C2', '9.1.4', 'F#3', '9.0.3.5', 'F4', '9.0.2', 'F3', 'D6', 'C#2', 'C4', 'D5', 'E6', 'E-5', 'D4', 'C3', 'E1', 'E2', 'F#4', 'E6', '5.7.0', 'B-4', 'E4', 'E3', '4.7.11', 'C#3', '5.9', '8.9.11', '9.11.0', 'E5', '8.0.2', 'B3', 'B2', 'F#3', 'E7', '7.11.2', 'C3', 'G#1', 'E-4', 'E-6', 'E3', 'E7', 'E2', 'F#4', 'F3', 'F#6', 'E-6', 'B-3', 'F#1', 'D4', 'F5', '6.8', 'E-4', 'F#2', 'E-3', '9.11.3', '9.1', 'C#2', '8.11.3', 'B1', '9.10', 'E-3', 'B-3', '9.11.2', 'G#5']
